# Transfer Learning Demo Using the VGG16 Model As A Base

In [1]:
# Load the Pre-trained VGG16 Model
# import VGG16 from keras.applications and load the model with ImageNet weights, excluding the top (fully connected) layers.

from tensorflow.keras.applications import VGG16
from tensorflow.image import resize
from cv2 import imread, imshow
import numpy as np
import os

# Load the VGG16 model, pre-trained weights, exclude top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [2]:
# Add Custom Layers
# Freeze the layers of the base model to prevent them from being updated during training, then add your custom layers on top for the classification task.
from tensorflow.keras import layers, models

# Freeze the base model
base_model.trainable = False

# Create a custom head for our network
model = models.Sequential([
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(1024, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(2, activation='softmax')  # Assuming two classes: Cat and Dog
])

In [3]:
# Compile and Train the Model
# Compile the model, specifying the loss function, optimizer, and metrics. Then, train the model with your dataset.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train/Test Split
In order to adapt the VGG16 to classify cats and dogs, we used a Kaggle dataset that can be found at https://www.kaggle.com/c/dogs-vs-cats. The train.zip was downloaded and extracted and is referenced here as the "dataset" folder. This set has a rigid naming scheme and 25000 photos of dogs and cats, with 12500 of each. Each filename has a structure of (animal).(number).jpg, so they can be parsed to create a train/test split. We made an 80% train, 20% test split by counting all the cat and dog images with numbers below 10000 as part of the training set, and all remaining 2500 images of each as part of the test set. Labels were assigned by determining if the filenames started with cat or dog. Before being added to a set, each image was resized to 224 by 224 to match the input of the VGG16 model.

In [4]:
train_images, test_images, train_labels, test_labels = [], [], [], []
DIR = 'dataset/'

def split_train_test():
    for root, dir, files in os.walk('./dataset'):
        for file in files:
            img = imread(DIR + file)
            processed = resize(
                np.expand_dims(img, axis=0), [224, 224]
            )
            cat_or_dog = 1 if file.startswith('cat') else 0
            # There are 12500 cats and dogs (indexed 0 to 12499) in the data set.
            # To make an 80%:20% train-test split, 10000 cats and 10000 dogs need to go into the train set.
            # All other cat and dog images go into the test set.
            if int(file[file.index('.')+1:file.rindex('.')]) < 10000:
                train_images.append(processed)
                train_labels.append(cat_or_dog)
            else:
                test_images.append(processed)
                test_labels.append(cat_or_dog)

split_train_test()

Here, a sample image and label are printed to make sure the data types in the sets are good. Each are converted to numpy arrays and reshaped as needed to match the VGG16 input shape.

In [5]:
# Take a sample of one of the photos and labels to make sure the format is correct
print(train_labels[1])
print(train_images[1])

# Convert labels to a numpy array, then convert image tensors of shape (x, 1, 224, 224, 3) to numpy arrays of shape (x, 224, 224, 3)
train_labels = np.asarray(train_labels)
train_images = np.array(train_images,dtype=np.float32).reshape((20000,224,224,3))
test_labels = np.asarray(test_labels)
test_images = np.array(test_images,dtype=np.float32).reshape((5000,224,224,3))

1
tf.Tensor(
[[[[ 40.125     44.125     39.125   ]
   [ 41.58817   44.061382  39.570312]
   [ 43.848213  44.848213  40.848213]
   ...
   [180.87604  206.7868   210.63501 ]
   [169.12376  201.10594  204.10594 ]
   [163.03015  197.69086  200.69086 ]]

  [[ 41.        45.        40.      ]
   [ 42.017857  44.49107   40.      ]
   [ 43.530132  44.530132  40.530132]
   ...
   [174.55463  200.35153  204.25667 ]
   [165.6651   196.71312  200.27228 ]
   [160.67636  194.21207  197.96207 ]]

  [[ 40.268974  44.268974  39.268974]
   [ 40.767857  43.24107   38.75    ]
   [ 41.219868  42.219868  38.219868]
   ...
   [168.77898  192.79578  197.3259  ]
   [164.99554  194.48663  198.97772 ]
   [165.66518  197.32588  202.32588 ]]

  ...

  [[ 28.785715  27.785715  29.785715]
   [ 25.725447  24.725447  26.725447]
   [ 21.830357  20.830357  22.830357]
   ...
   [ 25.214233  31.214233  44.214233]
   [ 21.107178  27.107178  39.125   ]
   [ 32.815792  38.815792  49.815792]]

  [[ 30.660715  29.660715  31.66

# Training and evaluating the augmented model
Finally, we train the model using keras' ```model.fit``` function with the training images and labels, and using the test sets as validation data. The model training allocated over 11 GiB of memory to run and took just over 10 minutes to complete one epoch. After one epoch, the training accuracy came out to be 96.46%, with test accuracy coming out to 97.92%.

In [6]:
# Assuming `train_images`, `train_labels` is the training dataset
# Evaluate the model assuming `test_images`, `test_labels` is the testing dataset
model.fit(train_images, train_labels, epochs=1, validation_data=(test_images,test_labels))




625/625 [==============================] - 785s 1s/step - loss: 0.2229 - accuracy: 0.9646 - val_loss: 0.0633 - val_accuracy: 0.9792


Once trained, we could now save the model.

In [7]:
# Save and Load Your Model
# Finally, save your trained model for later use and load it whenever you need to classify new images.
model.save('cats_vs_dogs_model.h5')

c:\Users\Anthony\Documents\CS Homework\AI 260 Deep Learning S2024\Transfer-Learning-6\deepLearning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Loading the model and making Analyses
#### We will now load and use the model to make a prediction on an image passed to it. This is done via the predict() function built into keras.

In [ ]:
# To load the model:
from tensorflow.keras.models import load_model

model = load_model('cats_vs_dogs_model.h5')

#### In the filepath variable below, the current configuration uses a string to represent a file being stored in the test folder. This string may be replaced with any filepath for the purpose of prediction.

In [16]:
# Open image to predict
filepath = "test/88.jpg"
img = imread(filepath)
processed_img = resize(np.expand_dims(img, axis=0), [224, 224])

#### An important note when defining classes for prediciton is that the order matters. In this particular instance, due to the way that dogs and cats were specified during training. 1 representing cats and 0 representing dogs, thus making dogs come first.

In [17]:
labels = ['dog','cat']
prediction = labels[model.predict(processed_img).argmax()]
print("The input image is a", prediction)

1/1 [==============================] - 0s 106ms/step
The input image is a dog
